# 验证集答案评价函数

In [ ]:
def eval_pre(answer_list):
    development_set_answers = open('./development_set_answers.txt').read()

    development_set_answers = development_set_answers.split('\n')

    development_set_answers = development_set_answers[:-1]

    for i in range(len(development_set_answers)):
        development_set_answers[i] = \
        development_set_answers[i][development_set_answers[i].find('[')+1:development_set_answers[i].find(']')]

    corr=0
    if len(answer_dic)!=len(development_set_answers):
        print('wrong')
    for i in range(len(answer_list)):
        if answer_list[i] == development_set_answers[i]:
            corr+=1
    print(corr,corr/len(answer_list))

# n-gram(2, 3, 4, 5) & 频率

## prepare

### 读取train set and dev set

In [ ]:
raw = open('./train-sentences.txt').read()
train_words = raw.split('\n')
train_words = train_words[:-1]

raw = open('./dev-sentences.txt').read()
answer_sentences = raw.split('\n')
answer_sentences = answer_sentences[:-1]

### 计算vocabular size

In [ ]:
import collections

# 计算vocabular size
raw = open('./train-sentences.txt').read()
raw = raw.replace('\n',' ')
raw = raw.split(' ')
counter = collections.Counter(raw)

voca_size = len(counter)
print(voca_size)

## n-gram-model

### 计算出现的频次

In [ ]:
import math

def n_gram(n,train_words):
    numerator = {}
    denominator = {}
    
    numerator_list = []
    denominator_list = []
    
    for i in range(len(train_words)):
        
        sentence = train_words[i].split(' ')
        denominator_list = []
        
        for k in range(n - 1):
            denominator_list.append('#')
        
        for k in range(0,len(sentence)):
            
            word_now = sentence[k]
            
            # 计算分母
            denominator_string = ''
            for word in denominator_list:
                denominator_string +=word
            
            denominator[denominator_string] = denominator.get(denominator_string,0) + 1
            
            # 计算分子
            numerator_string = denominator_string +  word_now
            numerator[numerator_string] = numerator.get(numerator_string,0) + 1
            
            # 更新
            for j in range(len(denominator_list) - 1):
                denominator_list[j] = denominator_list[j+1]
            denominator_list[-1] = word_now
        
        # 计算分母
        denominator_string = ''
        for word in denominator_list:
            denominator_string +=word
        denominator[denominator_string] = denominator.get(denominator_string,0) + 1
        
        # 计算分子
        word_now = '&'
        numerator_string = denominator_string +  word_now
        numerator[numerator_string] = numerator.get(numerator_string,0) + 1

        
    return numerator,denominator
  

### 得到每个句子出现的概率

In [ ]:
def get_pre(n,numerator,denominator,answer_sentences,voca_size,smooth):
    answer_dic = {}

    for i in range(len(answer_sentences)):
        
        if i%6 == 0:
            question_num = answer_sentences[i]
            answer_dic[question_num] = []

        else:
            sentence = answer_sentences[i].split(' ')
            pro = 0
            
            denominator_list = []
            
            # 前n-1个词，分母初始化
            
            for k in range(n - 1):
                denominator_list.append('#')


            for k in range(0,len(sentence)):
                
                word_now = sentence[k]
                
                # 计算分母
                denominator_string = ''
                for word in denominator_list:
                    denominator_string += word
                    
                # 分子字符串
                numerator_string = denominator_string+word_now
                
                #print(numerator.get(numerator_string,0),(denominator.get(denominator_string,0)))
                
                pro += math.log((numerator.get(numerator_string,0)+smooth)/(denominator.get(denominator_string,0)+smooth*voca_size))
                
                # 更新
                for j in range(len(denominator_list) - 1):
                    denominator_list[j] = denominator_list[j+1]
                denominator_list[-1] = word_now
                
            # 计算分母
            denominator_string = ''
            for word in denominator_list:
                denominator_string +=word
            
            # 计算分子
            word_now = '&'
            numerator_string = denominator_string +  word_now
            pro += \
                math.log((numerator.get(numerator_string,0)+smooth)/(denominator.get(denominator_string,0)+smooth))

            answer_dic[question_num].append(pro)
            
    answer_list = []
    compare = ['a','b','c','d','e']

    for k,v in answer_dic.items():
        maxIndex = v.index(max(v))
        answer_list.append(str(compare[maxIndex]))
            
    return answer_dic,answer_list

In [ ]:
smooth = 1
print('smooth: ',smooth)

for n in range(2,6):
    numerator,denominator = n_gram(n,train_words)
    answer_dic,answer_list = get_pre(n,numerator,denominator,answer_sentences,voca_size,smooth)
    print(str(n)+'-gram:')
    print('概率:',end='')
    eval_pre(answer_list)
    answer_dic,answer_list = get_pre(n,numerator,{},answer_sentences,1,smooth)
    print('频率:',end='')
    eval_pre(answer_list)

smooth = 0.5
print('smooth: ',smooth)
print()

for n in range(2,6):
    numerator,denominator = n_gram(n,train_words)
    answer_dic,answer_list = get_pre(n,numerator,denominator,answer_sentences,voca_size,smooth)
    print(str(n)+'-gram:')
    print('概率:',end='')
    eval_pre(answer_list)
    answer_dic,answer_list = get_pre(n,numerator,{},answer_sentences,1,smooth)
    print('频率:',end='')
    eval_pre(answer_list)

# gensim

## gensim word2vec score

### 训练词向量 参数 (hs=1,negative=0,sample=0.001,window=2,size=200,min_count=10,workers =4)

In [ ]:
## gensim

from gensim.models import word2vec
from gensim.models import KeyedVectors

# 引入日志配置
#import logging

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 引入数据集
raw_sentences = open('./train-sentences.txt').read()



raw_sentences = raw_sentences.split('\n')

# 切分词汇
sentences = [s.split(' ') for s in raw_sentences]

# 构建word2vec
model = word2vec.Word2Vec(sentences,hs=1,negative=0,sample=0.001,window=10,size=300,min_count=10,workers = 7)

### 读取dev set

In [ ]:
# 引入数据集
raw_sentences = open('./dev-sentences.txt').read()
answer_sentences = raw_sentences.split('\n')
answer_sentences = answer_sentences[:-1]

### 得到score

In [ ]:
def get_pre_wv(answer_sentences,model):
    answer_dic = {}

    for i in range(len(answer_sentences)):
        
        if i%6 == 0:
            question_num = answer_sentences[i]
            answer_dic[question_num] = []

        else:
            sentence = answer_sentences[i].split(' ')
            answer_dic[question_num].append(model.score([answer_sentences[i].split(' ')]))
            
            
    answer_list = []
    compare = ['a','b','c','d','e']
    

    for k,v in answer_dic.items():
        maxIndex = v.index(max(v))
        answer_list.append(str(compare[maxIndex]))
            
    return answer_dic,answer_list

answer_dic,answer_list = get_pre_wv(answer_sentences,model)

### dev正确率

In [ ]:
eval_pre(answer_list)

## gensim word2vec predict_output_word

### 训练词向量 参数 (hs=1,window=7,size=300,min_count=1,workers =4)

In [ ]:
from gensim.models import word2vec
from gensim.models import KeyedVectors

# 引入日志配置
#import logging

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 引入数据集
raw_sentences = open('./train-sentences.txt').read()



raw_sentences = raw_sentences.split('\n')

# 切分词汇
sentences = [s.split(' ') for s in raw_sentences]

# 构建模型
model = word2vec.Word2Vec(sentences,hs=1,window=10,size=300,min_count=5,workers =7)

### 准备上下文和答案

In [ ]:
import re
dev = open('./development_set.txt').read()

dev = re.sub(r"\n\n\n",'#', dev)

dev_questions = dev.split('#')
dev_questions = dev_questions[0:-1]


answer_abcde_dic = {}
question_dic = {}

for dev_question in dev_questions:
    
    question_num = dev_question[:dev_question.find(')')]
    question = dev_question[dev_question.find(' ')+1:dev_question.find('\n')]
    
    
    
    a_answer = dev_question[dev_question.find('a)')+3:dev_question.find('\n',dev_question.find('a)')+3)]
    b_answer = dev_question[dev_question.find('b)')+3:dev_question.find('\n',dev_question.find('b)')+3)]
    c_answer = dev_question[dev_question.find('c)')+3:dev_question.find('\n',dev_question.find('c)')+3)]
    d_answer = dev_question[dev_question.find('d)')+3:dev_question.find('\n',dev_question.find('d)')+3)]
    e_answer = dev_question[dev_question.find('e)')+3:]
    
    question_dic[question_num] = question.replace('_____','')
    
    answer_abcde_dic[question_num] = []
    answer_abcde_dic[question_num].append(a_answer)
    answer_abcde_dic[question_num].append(b_answer)
    answer_abcde_dic[question_num].append(c_answer)
    answer_abcde_dic[question_num].append(d_answer)
    answer_abcde_dic[question_num].append(e_answer)
    
import string
for key in question_dic.keys():
    for letter in string.punctuation:
        question_dic[key] = question_dic[key].replace(letter,' ')
        
# 多个' '变一个' '，去掉前后的' '
for key in question_dic.keys():
    question_dic[key] = re.sub(' +',' ',question_dic[key])
    question_dic[key] = question_dic[key].strip() 
# 变成小写
for key in question_dic.keys():
    question_dic[key] = question_dic[key].lower()

### 得到预测的概率

In [ ]:
def get_pre_wv_pro(question_dic,answer_abcde_dic,model):
    answer_dic = {}

    for key in question_dic.keys():
        
        answer_dic[key] = []
        
        tupeList = model.predict_output_word(question_dic[key].split(' '),topn=40000)
        dicList = {}
        for tup in tupeList:
            dicList[tup[0]] = tup[1]
        
        
        for answer in answer_abcde_dic[key]:
            pro = 0
            if answer in dicList:
                pro = dicList[answer]
                
            answer_dic[key].append(pro)
            
        
            
    answer_list = []
    compare = ['a','b','c','d','e']

    for k,v in answer_dic.items():
        maxIndex = v.index(max(v))
        answer_list.append(str(compare[maxIndex]))
            
    return answer_dic,answer_list


answer_dic,answer_list = get_pre_wv_pro(question_dic,answer_abcde_dic,model)


### 评价

In [ ]:
eval_pre(answer_list)

## 相似度比较

### 训练词向量 参数 (hs=1,window=7,size=300,min_count=1,workers =4)

In [ ]:
from gensim.models import word2vec
from gensim.models import KeyedVectors

# 引入日志配置
#import logging

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 引入数据集
raw_sentences = open('./train-sentences.txt').read()



raw_sentences = raw_sentences.split('\n')

# 切分词汇
sentences = [s.split(' ') for s in raw_sentences]

# 构建模型
model = word2vec.Word2Vec(sentences,hs=1,window=10,size=300,min_count=5,workers =7)

### 准备上下文和答案

In [ ]:
import re
dev = open('./development_set.txt').read()

dev = re.sub(r"\n\n\n",'#', dev)

dev_questions = dev.split('#')
print(len(dev_questions))
dev_questions = dev_questions[0:-1]
print(len(dev_questions))


answer_abcde_dic = {}
question_dic = {}

for dev_question in dev_questions:
    
    question_num = dev_question[:dev_question.find(')')]
    question = dev_question[dev_question.find(' ')+1:dev_question.find('\n')]
    
    
    
    a_answer = dev_question[dev_question.find('a)')+3:dev_question.find('\n',dev_question.find('a)')+3)]
    b_answer = dev_question[dev_question.find('b)')+3:dev_question.find('\n',dev_question.find('b)')+3)]
    c_answer = dev_question[dev_question.find('c)')+3:dev_question.find('\n',dev_question.find('c)')+3)]
    d_answer = dev_question[dev_question.find('d)')+3:dev_question.find('\n',dev_question.find('d)')+3)]
    e_answer = dev_question[dev_question.find('e)')+3:]
    
    question_dic[question_num] = question.replace('_____','')
    
    answer_abcde_dic[question_num] = []
    answer_abcde_dic[question_num].append(a_answer)
    answer_abcde_dic[question_num].append(b_answer)
    answer_abcde_dic[question_num].append(c_answer)
    answer_abcde_dic[question_num].append(d_answer)
    answer_abcde_dic[question_num].append(e_answer)

import string
for key in question_dic.keys():
    for letter in string.punctuation:
        question_dic[key] = question_dic[key].replace(letter,' ')
        
# 多个' '变一个' '，去掉前后的' '
for key in question_dic.keys():
    question_dic[key] = re.sub(' +',' ',question_dic[key])
    question_dic[key] = question_dic[key].strip() 
# 变成小写
for key in question_dic.keys():
    question_dic[key] = question_dic[key].lower()

In [ ]:
def get_total_similarity(question_dic,answer_dic,model):
    answer_dic = {}

    for key,v in question_dic.items():

        answer_dic[key] = []

        for answer in answer_abcde_dic[key]:

            similarity = 0
            words = v.split(' ')
            for word in words:
                if word in model and answer in model:
                    similarity+=model.similarity(answer,word)
            answer_dic[key].append(similarity)
    answer_list = []
    compare = ['a','b','c','d','e']

    for k,v in answer_dic.items():
        maxIndex = v.index(max(v))
        answer_list.append(str(compare[maxIndex]))

    return answer_dic,answer_list

In [ ]:
answer_dic,answer_list = get_total_similarity(question_dic,answer_abcde_dic,model)

### 评价

In [ ]:
eval_pre(answer_list)